In [1]:
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists
from urllib.parse import quote_plus as urlquote
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Load Datasets

In [2]:
#basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [3]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)

In [4]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [5]:
#ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [6]:
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)

In [7]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1962
1,tt0000002,5.8,263
2,tt0000005,6.2,2601
3,tt0000006,5.1,178
4,tt0000007,5.4,817


In [8]:
#akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [9]:
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)

In [10]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


# Data Cleaning

## Basics Dataset

In [11]:
basics = basics.replace({'\\N':np.nan})

In [12]:
basics = basics.dropna(subset=['runtimeMinutes','genres','startYear'])

In [13]:
basics = basics.loc[basics['titleType'] == 'movie']

In [14]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [15]:
basics['startYear'] = basics['startYear'].astype(int)

In [16]:
#Something is wrong here
basics = basics.loc[basics['startYear'] >= 2000]

In [17]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

## Akas Dataset

In [18]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [19]:
akas = akas.loc[akas['region'] == 'US']

In [20]:
akas = akas.replace({'\\N':np.nan})

## Ratings Dataset

In [21]:
ratings = ratings.replace({'\\N':np.nan})

## Keep only US movies

In [22]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

0        True
1        True
2        True
3        True
4        True
         ... 
87295    True
87296    True
87297    True
87298    True
87299    True
Name: tconst, Length: 87300, dtype: bool

In [23]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
87295,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
87296,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
87297,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
87298,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [24]:
# Filter the ratings table down to only include the US by using the filter akas dataframe
keepers =ratings['tconst'].isin(akas['titleId'])
keepers

0         True
1         True
2         True
3         True
4         True
          ... 
493041    True
493042    True
493043    True
493044    True
493045    True
Name: tconst, Length: 493046, dtype: bool

In [25]:
ratings = ratings[keepers]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1962
1,tt0000002,5.8,263
2,tt0000005,6.2,2601
3,tt0000006,5.1,178
4,tt0000007,5.4,817
...,...,...,...
493041,tt9916200,8.2,225
493042,tt9916204,8.2,257
493043,tt9916348,8.3,18
493044,tt9916362,6.4,5244


# Saving Filtered Datasets

In [26]:
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['title_basics.csv.gz', 'title_akas.csv.gz', 'title_ratings.csv.gz']

In [27]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [28]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [29]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87300 entries, 0 to 87299
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          87300 non-null  object 
 1   titleType       87300 non-null  object 
 2   primaryTitle    87300 non-null  object 
 3   originalTitle   87300 non-null  object 
 4   isAdult         87300 non-null  int64  
 5   startYear       87300 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  87300 non-null  int64  
 8   genres          87300 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 6.0+ MB


In [30]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [31]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [32]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424311 entries, 0 to 1424310
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1424311 non-null  object 
 1   ordering         1424311 non-null  int64  
 2   title            1424311 non-null  object 
 3   region           1424311 non-null  object 
 4   language         3867 non-null     object 
 5   types            976245 non-null   object 
 6   attributes       46253 non-null    object 
 7   isOriginalTitle  1422966 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 86.9+ MB


In [33]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [34]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1962
1,tt0000002,5.8,263
2,tt0000005,6.2,2601
3,tt0000006,5.1,178
4,tt0000007,5.4,817


In [35]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493046 entries, 0 to 493045
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         493046 non-null  object 
 1   averageRating  493046 non-null  float64
 2   numVotes       493046 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.3+ MB
